In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline

In [13]:
df1 = pd.read_excel("../data/data1.xlsx", sheet_name="payment")
df2 = pd.read_excel("../data/data1.xlsx")

In [14]:
# pivot table
df_2 = df.groupby("user_id")["payment"].agg(["count", "sum"]).reset_index()

# rename
df_2.rename({"count":"payment_count", "sum":"sales"}, axis = 1, inplace = True)

In [17]:
# rolling retention 구하기
# sign_up month만 가져오기
df2["sign_up_month"] = df2["sign_up"].dt.month
df2["last_login_month"] = df2["last_login"].dt.month

In [23]:
# 병합
df_2 = pd.merge(df_2, df2[["user_id", "sign_up_month"]], how = "left", on="user_id")

In [26]:
# pivot_table
# user_id : 달에 가입한 사람
# sales : 달에 가입한 사람이 12월에 결제한 매출
df_2_pivot = df_2.groupby("sign_up_month")["sales", "user_id"].agg({"user_id":"count", "sales":"sum"})

In [45]:
df_2_pivot.rename({"user_id":"결제자수", "sales": "매출"}, axis = 1,inplace = True)

In [42]:
# pivot table 월별 가입자 수 구하기
retention_df = pd.pivot_table(data=df2, index="sign_up_month",
               columns="last_login_month", values="user_id", aggfunc = "count")
retention_df["합계"] = retention_df.sum(axis = 1)
retention_df.fillna(0, inplace = True)

df_2_pivot["가입자"] =retention_df["합계"]
df_2_pivot["활동회원"] = retention_df[12]

In [37]:
# 활동회원 구하기
for idx in range(len(retention_df)):
    num = retention_df.iloc[idx, -1]
    for month in range(idx +1, 13):
        month_num = retention_df.loc[idx + 1, month]
        retention_df.loc[idx + 1, month] = num
        num -= month_num


import copy
retention_df2 = copy.deepcopy(retention_df)

for i in range(len(retention_df2)):
    a = retention_df2.loc[i + 1, i+1]
    month += 1
    for col in range(1, 13):
        retention_df2.loc[i+1, col] = retention_df2.loc[i+1, col] / a

In [43]:
df_2_pivot["잔존율"] = retention_df2[12]

In [48]:
df_2_pivot

,결제자수,매출,가입자,활동회원,잔존율
sign_up_month,,,,,
1,25,221000,428.0,59.0,0.137850
2,39,301000,481.0,111.0,0.230769
3,57,496000,545.0,171.0,0.313761
4,49,495000,501.0,184.0,0.367265
5,58,542000,530.0,238.0,0.449057
6,65,597000,530.0,291.0,0.549057
7,70,643000,495.0,308.0,0.622222
8,72,634000,534.0,348.0,0.651685
9,99,1175000,632.0,473.0,0.748418


In [50]:
# 정렬
df_2_pivot = df_2_pivot[["가입자", "잔존율", "활동회원", "결제자수", "매출"]]

In [51]:
# 결제비율
df_2_pivot["결제비율"] = df_2_pivot["결제자수"] / df_2_pivot["활동회원"]

In [54]:
# arppu
df_2_pivot["ARPPU"] = round(df_2_pivot["매출"] / df_2_pivot["결제자수"])

In [57]:
# 정렬
df_2_pivot = df_2_pivot[["가입자", "잔존율", "활동회원", "결제비율", "결제자수", "ARPPU", "매출"]]

In [68]:
df_2_pivot.style.background_gradient("summer")

,가입자,잔존율,활동회원,결제비율,결제자수,ARPPU,매출
sign_up_month,,,,,,,
1,428.000000,0.137850,59.000000,0.423729,25,8840.000000,221000
2,481.000000,0.230769,111.000000,0.351351,39,7718.000000,301000
3,545.000000,0.313761,171.000000,0.333333,57,8702.000000,496000
4,501.000000,0.367265,184.000000,0.266304,49,10102.000000,495000
5,530.000000,0.449057,238.000000,0.243697,58,9345.000000,542000
6,530.000000,0.549057,291.000000,0.223368,65,9185.000000,597000
7,495.000000,0.622222,308.000000,0.227273,70,9186.000000,643000
8,534.000000,0.651685,348.000000,0.206897,72,8806.000000,634000
9,632.000000,0.748418,473.000000,0.209302,99,11869.000000,1175000


In [69]:
df_2_pivot.to_csv("../data/data2.csv")